In [1]:
import pandas as pd
import matplotlib.pyplot as mpl
import numpy as np
import os
from datetime import datetime
import plotly.express as px
from Minu_Timeseries import timeseries_withdrawl

mpl.rcParams['figure.figsize'] = (10,8)
mpl.rcParams['axes.grid'] = False

In [2]:
# Pre-processing using my package

Excel_Name = 'ac_statement.xlsx'
Sheet_Name = 'DS_1'
df1 = timeseries_withdrawl(Excel_Name, Sheet_Name, False)


In [5]:
df1 = df1.reset_index().rename(columns={df1.index.name:'Date'})

In [6]:
df1.head(2)

,Date,Withdrawl_Amount
0,2015-01-01,0.00
1,2015-01-02,5500056.18


In [10]:
df1

,Date,Withdrawl_Amount
0,2015-01-01,0.00
1,2015-01-02,5500056.18
2,2015-01-03,2200000.00
3,2015-01-04,0.00
4,2015-01-05,3231000.00
...,...,...
1520,2019-03-01,4532396.00
1521,2019-03-02,12633316.00
1522,2019-03-03,0.00
1523,2019-03-04,4800000.00


In [14]:
min_d = min(df1['Date'])
max_d = max(df1['Date'])

In [15]:
withdrawl_trans = df1[df1['Withdrawl_Amount'] > 0]

In [16]:
withdrawl_trans

,Date,Withdrawl_Amount
1,2015-01-02,5500056.18
2,2015-01-03,2200000.00
4,2015-01-05,3231000.00
5,2015-01-06,3880056.18
6,2015-01-07,3650000.00
...,...,...
1519,2019-02-28,6498321.00
1520,2019-03-01,4532396.00
1521,2019-03-02,12633316.00
1523,2019-03-04,4800000.00


In [17]:
q95_2015 = withdrawl_trans['Withdrawl_Amount'][withdrawl_trans['year'] == 2015].quantile(.95)
q05_2015 = withdrawl_trans['Withdrawl_Amount'][withdrawl_trans['year'] == 2015].quantile(.05)


KeyError: 'year'

In [ ]:
#treating outliers --Need to make loop for this
q95_2015 = withdrawl_trans['Withdrawl_Amount'][withdrawl_trans['year'] == 2015].quantile(.95)
q05_2015 = withdrawl_trans['Withdrawl_Amount'][withdrawl_trans['year'] == 2015].quantile(.05)

q95_2016 = withdrawl_trans['Withdrawl_Amount'][withdrawl_trans['year'] == 2016].quantile(.95)
q05_2016 = withdrawl_trans['Withdrawl_AmountT'][withdrawl_trans['year'] == 2016].quantile(.05)

q95_2017 = withdrawl_trans['Withdrawl_Amount'][withdrawl_trans['year'] == 2017].quantile(.95)
q05_2017 = withdrawl_trans['Withdrawl_Amount'][withdrawl_trans['year'] == 2017].quantile(.05)

q95_2018 = withdrawl_trans['Withdrawl_Amount'][withdrawl_trans['year'] == 2018].quantile(.95)
q05_2018 = withdrawl_trans['Withdrawl_Amount'][withdrawl_trans['year'] == 2018].quantile(.05)

q95_2019 = withdrawl_trans['Withdrawl_Amount'][withdrawl_trans['year'] == 2019].quantile(.95)
q05_2019 = withdrawl_trans['Withdrawl_Amount'][withdrawl_trans['year'] == 2019].quantile(.05)

withdrawl_trans.loc[ (withdrawl_trans['year'] == 2015) &  (withdrawl_trans['Withdrawl_Amount'] <= q05_2015 ),'Withdrawl_Amount' ]=  q05_2015  
withdrawl_trans.loc[ (withdrawl_trans['year'] == 2015)  & (withdrawl_trans['Withdrawl_Amount'] > q95_2015) ,'Withdrawl_Amount'] =  q95_2015       

withdrawl_trans.loc[(withdrawl_trans['year'] == 2016) & ( withdrawl_trans['Withdrawl_Amount'] <= q05_2016) ,'Withdrawl_Amount']=  q05_2016  
withdrawl_trans.loc[(withdrawl_trans['year'] == 2016) &  (withdrawl_trans['Withdrawl_Amount'] > q95_2016 ),'Withdrawl_Amount'] =  q95_2016 

withdrawl_trans.loc[(withdrawl_trans['year'] == 2017) &  (withdrawl_trans['Withdrawl_Amount'] <= q05_2017) ,'Withdrawl_Amount'] =  q05_2017  
withdrawl_trans.loc[(withdrawl_trans['year'] == 2017) &  (withdrawl_trans['Withdrawl_Amount'] > q95_2017),'Withdrawl_Amount'] =  q95_2017  

withdrawl_trans.loc[(withdrawl_trans['year'] == 2018) &  (withdrawl_trans['Withdrawl_Amount'] <= q05_2018) ,'Withdrawl_Amount'] =  q05_2018  
withdrawl_trans.loc[(withdrawl_trans['year'] == 2018) &  (withdrawl_trans['Withdrawl_Amount'] > q95_2018),'Withdrawl_Amount'] =  q95_2018 

withdrawl_trans.loc[(withdrawl_trans['year'] == 2019) &  (withdrawl_trans['Withdrawl_Amount'] <= q05_2019) ,'Withdrawl_Amount'] =  q05_2019  
withdrawl_trans.loc[(withdrawl_trans['year'] == 2019) &  (withdrawl_trans['Withdrawl_Amount'] > q95_2019),'Withdrawl_Amount'] =  q95_2019        


In [ ]:
df1 = (deposit_trans.groupby(['DATE'])['DEPOSIT AMT'].sum().reset_index(name='total_dep'))

In [ ]:
#df1.to_csv("clubbed_data.csv")
day_1 = min(df['DATE'])
day_n = max(df['DATE'])
daterange = pd.DataFrame({'DATE': pd.date_range(start = day_1, end = day_n)})
df_deposit = pd.merge(daterange,df1,on='DATE',how='left')
df_agg = df.groupby(['DATE'])['cash_deposit','cheq_deposit','Neft_trans', 'Rtgs_trans', 'cash_pickup','Internal_fund_transfer', 'IMPS_trans', 'TRF_trans','indiaforensic_trans', 'DSB_trans', 'Charges_tax', 'other_trans'].max()
holidays_list = df_deposit[df_deposit['total_dep'].isnull()]
holidays_list['total_dep'] = 'sunday_holiday'
holidays_list.columns = ['ds','holiday']

df_deposit['total_dep'] = df_deposit['total_dep'].fillna(100)
data_final =  pd.merge(df_deposit,df_agg,on='DATE',how='left')
data_final['dayofweek'] = data_final['DATE'].dt.dayofweek
data_final['quarter'] = data_final['DATE'].dt.quarter
data_final['month'] = data_final['DATE'].dt.month
data_final['year'] = data_final['DATE'].dt.year
data_final['dayofyear'] = data_final['DATE'].dt.dayofyear
data_final['dayofmonth'] = data_final['DATE'].dt.day
data_final['weekofyear'] = data_final['DATE'].dt.weekofyear
data_final.fillna(0,inplace=True)

In [ ]:
data_final.head(2)

In [ ]:
from fbprophet import Prophet

mydata = data_final.rename({'DATE': 'ds', 'total_dep': 'y'},axis=1)
train = mydata[(mydata['ds'] >='2015-01-01' ) & (mydata['ds'] <'2018-01-01' )]
test =  mydata[(mydata['ds'] >='2018-01-01' )]
print(train.shape)
print(test.shape)

In [ ]:
train

In [ ]:

'''
m.add_regressor('cash_deposit', standardize=True) 
m.add_regressor('cheq_deposit',standardize=True)
m.add_regressor('Neft_trans',standardize=True)
m.add_regressor('Rtgs_trans',standardize=True)
m.add_regressor('cash_pickup',standardize=True)
m.add_regressor('Internal_fund_transfer',standardize=True)
m.add_regressor('IMPS_trans',standardize=True)
m.add_regressor('TRF_trans',standardize=True)
m.add_regressor('DSB_trans',standardize=True)
m.add_regressor('Charges_tax',standardize=True)
m.add_regressor('other_trans',standardize=True)
m.add_regressor('dayofweek',standardize=True)
m.add_regressor('quarter',standardize=True)
m.add_regressor('month',standardize=True)
m.add_regressor('year',standardize=True)
m.add_regressor('dayofyear',standardize=True)
m.add_regressor('dayofmonth',standardize=True)
m.add_regressor('weekofyear',standardize=True)
#m.fit(train)
m.params
 '''

In [ ]:
m = Prophet(interval_width=0.95)
m.fit(train)

In [ ]:
future = m.make_future_dataframe(periods = 429)
future.tail()
future['cash_deposit'] = mydata['cash_deposit']
future['cheq_deposit'] = mydata['cheq_deposit']
future['Neft_trans'] = mydata['Neft_trans']
future['Rtgs_trans'] = mydata['Rtgs_trans']
future['cash_pickup'] = mydata['cash_pickup']
future['Internal_fund_transfer'] = mydata['Internal_fund_transfer']
future['IMPS_trans'] = mydata['IMPS_trans']
future['TRF_trans'] = mydata['TRF_trans']
future['DSB_trans'] = mydata['TRF_trans']
future['Charges_tax'] = mydata['TRF_trans']
future['other_trans'] = mydata['TRF_trans']
future['dayofweek'] = mydata['dayofweek']
future['quarter'] = mydata['quarter']
future['month'] = mydata['month']
future['year'] = mydata['year']
future['dayofyear'] = mydata['dayofyear']
future['dayofmonth'] = mydata['dayofmonth']
future['weekofyear'] = mydata['weekofyear']
forecast = m.predict(future)

fig1 = m.plot(forecast)
fig1.show()
fig2 = m.plot_components(forecast)

from fbprophet.diagnostics import cross_validation, performance_metrics
cv_results = cross_validation(model=m, initial='731 days', horizon = '45 days')
df_p = performance_metrics(cv_results)
#df_p.to_csv("removed outliers_and weekly seasonality.csv")
#forecast.to_csv("final_predictions_outliers.csv",index=False)
#df_deposit.to_csv("actual_deposits.csv",index=False)

from fbprophet.plot import  plot_cross_validation_metric
fig3 = plot_cross_validation_metric(cv_results, metric= 'mape')
pd.concat([df_deposit.set_index('DATE')['total_dep'],forecast.set_index('ds')['yhat']],axis=1).plot()

#univariate prophet
df_deposit['ds']=pd.to_datetime(df_deposit['ds'])

import plotly.express as px
fig = px.line(df_deposit, x='ds', y='y', title='deposit transactions')
fig.update_xaxes(rangeslider_visible=True,rangeselector=dict(buttons=list([dict(count=1, label="1y", step="year", stepmode="backward"),dict(count=2, label="3y", step="year", stepmode="backward"),dict(count=3, label="5y", step="year", stepmode="backward"),dict(step="all")])))
fig.show()

df_deposit_new = df_deposit.set_index('ds')
df_deposit_new.plot()
df_deposit.reset_index()
df_deposit.columns =  ['ds', 'y']
train=df_deposit[(df_deposit['ds'] >= '2015-01-01' ) & (df_deposit['ds'] <= '2018-01-01' )]
test=df_deposit[(df_deposit['ds'] > '2018-01-01')]
print(train.shape)
print(test.shape)
model = Prophet()
model.fit(train)
future_uni = model.make_future_dataframe(periods=428)
forecast_uni = model.predict(future_uni)
forecast_uni
pd.concat([df_deposit.set_index('ds')['y'],forecast_uni.set_index('ds')['yhat']],axis=1).plot()
fig1 = model.plot(forecast_uni)
fig2 = model.plot_components(forecast_uni)

from fbprophet.plot import add_changepoints_to_plot
fig = model.plot(forecast_uni)
a = add_changepoints_to_plot(fig.gca(), model, forecast_uni)
deltas = model.params['delta'].mean(0)
deltas
fig
fig = plt.figure(facecolor='w')
ax = fig.add_subplot(111)
ax.bar(range(len(deltas)), deltas)
ax.grid(True, which='major', c='red', ls='-', lw=1, alpha=0.2)
ax.set_ylabel('Rate change')
ax.set_xlabel('changepoint')
fig.tight_layout()
fig

#changing change points to cover 90% of the data
pro_change= Prophet(changepoint_range = 0.9, yearly_seasonality=True,holidays= holidays_list)
forecast_uni = pro_change.fit(train).predict(future_uni)
fig= pro_change.plot(forecast_uni);
a = add_changepoints_to_plot(fig.gca(), pro_change, forecast_uni)
fig
 
from fbprophet.diagnostics import cross_validation
cv_results = cross_validation( pro_change, initial = '731 days', horizon = '43 days')

from fbprophet.diagnostics import performance_metrics
df_p = performance_metrics(cv_results)
df_p

 

#df_p.to_csv("third_evaluation.csv")